In [1]:
# 먼저 해당 라이브러리를 받아옵니다.
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random
from random import randint
from time import sleep
from tqdm import tqdm_notebook
import pandas as pd

In [2]:
def get_news(n_url):
    news_detail = []
    print(n_url)
    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    # html 파싱
    title = bsoup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    # 날짜 파싱
    pdate = bsoup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate)

    # 기사 본문 크롤링 
    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())

    # 신문사 크롤링
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)

    return news_detail


In [3]:
query_list = pd.read_excel('../data/검색어.xlsx')
query_list = query_list['검색어'].tolist()

query_list = pd.read_excel('../data/검색어_regex_2020.xlsx')
query_list = query_list[query_list['여부'] == 1]['검색어'].tolist()
'.'.join(query_list)

In [7]:
import os
os.makedirs('../crawling/', exist_ok = True)

#query_list = ['식품, 살모넬라','중금속']
query_list = ['안전']

In [14]:
set(query_list) - set(aa)

{'AI',
 'BSE',
 '곰팡이독소',
 '광우병',
 '기생충',
 '농약',
 '동물용의약품',
 '발암',
 '수의약품',
 '오염',
 '이물',
 '조류인플루엔자',
 '중금속',
 '첨가물'}

In [16]:
query_list = list(set(query_list) - set(aa))

In [4]:
query_list

['오염',
 '농약',
 '중금속',
 '기생충',
 '동물용의약품',
 '수의약품',
 '이물',
 'AI',
 '조류인플루엔자',
 '첨가물',
 '곰팡이독소',
 'BSE',
 '광우병',
 '항생제',
 '발암',
 '식중독',
 'E.coli',
 '다이옥신',
 '식품알레르기',
 '살모넬라',
 '멜라민',
 '방사능',
 '리스테리아',
 '비스페놀A',
 'GMO',
 '유전자변형']

In [3]:
# 쿼리에 검색어를 입력하고 검색 시작날짜부터 끝 날짜까지를 입력
s_date = "2017.01.01" # yyyy.mm.dd로 입력 요망
e_date = "2022.06.01"
s_from = s_date.replace(".","")
e_to = e_date.replace(".","")
    
# 최대 몇개까지 크롤링을 할찌 숫자를 입력

n_page = 5000

In [9]:
for q in tqdm_notebook(query_list) :

    page = 1

    # 저장 경로를 입력합니다.
    f = open("../crawling/" + s_date + q + '.txt', 'w', encoding='utf-8')


    while page < n_page:
        print(page)
        url = "https://search.naver.com/search.naver?where=news&query=식품," + q + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        #header 추가
        header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        }
        req = requests.get(url,headers=header)
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
            #print(soup)

        for urls in soup.select("._sp_each_url"):
            try :
                #print(urls["href"])
                if urls["href"].startswith("https://news.naver.com"):
                    #print(urls["href"])
                    news_detail = get_news(urls["href"])
                        # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[3], news_detail[0],
                                                          news_detail[2]))  # new style
            except Exception as e:
                print(e) 
                continue
        page += 10  
        sleep(2+ randint(1,100)/50)
    f.close()

<ipython-input-9-ceb237de6cd2>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for q in tqdm_notebook(query_list) :


  0%|          | 0/1 [00:00<?, ?it/s]

1
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=1
11
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=11
21
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=21
31
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=31
41
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=41
51
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=51
61
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&d

531
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=531
541
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=541
551
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=551
561
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=561
571
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=571
581
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=581
591
https://search.naver.com/search.naver?where=news&query

1061
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=1061
1071
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=1071
1081
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=1081
1091
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=1091
1101
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=1101
1111
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=1111
1121
https://search.naver.com/search.naver?whe

...

2621
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=2621
2631
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=2631
2641
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=2641
2651
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=2651
2661
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=2661
2671
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=2671
2681
https://search.naver.com/search.naver?whe

3141
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3141
3151
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3151
3161
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3161
3171
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3171
3181
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3181
3191
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3191
3201
https://search.naver.com/search.naver?whe

3661
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3661
3671
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3671
3681
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3681
3691
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3691
3701
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3701
3711
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=3711
3721
https://search.naver.com/search.naver?whe

4181
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4181
4191
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4191
4201
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4201
4211
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4211
4221
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4221
4231
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4231
4241
https://search.naver.com/search.naver?whe

4701
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4701
4711
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4711
4721
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4721
4731
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4731
4741
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4741
4751
https://search.naver.com/search.naver?where=news&query=식품,안전&sort=0&ds=2017.01.01&de=2022.06.01&nso=so%3Ar%2Cp%3Afrom20170101to20220601%2Ca%3A&start=4751
4761
https://search.naver.com/search.naver?whe